# Phase 3 timeplot

To plot the flagging percentages over time

As a waterfall plot? Then I can fit plot frequency vs time and color as flagging percentage.



In [1]:
#imports
import math
import numpy as np
import pandas as pd
import os
import sys
import subprocess
import matplotlib as plt
import seaborn as sns
%matplotlib qt

## PHASE 3

In [2]:
#set to load in xlsx files(True) or txt files(False). Should return the same result either way.
xlsx=False

#load in files
files_raw = str(subprocess.check_output(['ls', 'Phase3/']))
files = files_raw.split('\\n')

#fix files
files[0] = files[0][2:]#remove "b' from beginning
files=files[:-1]#remove non-entry last item

filelist=[]
#get rid of xlsx files from file selection
if xlsx==False:
    for filename in files:
        if filename[-4:] == '.txt':
            filelist.append(filename)
#get rid of txt files from file selection
if xlsx==True:
    for filename in files:
        if filename[-4:] == 'xlsx':
            filelist.append(filename)
            
#last three observations have fucked frequencies, so getting rid of them here:
#filelist=filelist[:-3]
        
#manipulate names in files, create observations list
observations = []
for i in range(0,len(filelist)):
    if xlsx==False:
        observations.append(filelist[i][8:-15])
    if xlsx==True:
        observations.append(filelist[i][:-5])

obs_dfs = {}#dictionary of dataframes

#read in files
for obs in observations:
    if obs not in obs_dfs.keys():
        if xlsx==False:
            obs_dfs[obs] = pd.read_table('Phase3/13B-266.'+obs+'.flag_stats.txt',sep=' ')
            #fix each dataframe
            obs_dfs[obs].drop(['#total','%flagged'],axis=1,inplace=True)
            obs_dfs[obs].columns = ['freq','#flags','#total','%flagged']
            obs_dfs[obs].set_index('freq',inplace=True)
        if xlsx==True:
            obs_dfs[obs] = pd.read_excel('Phase3/'+obs+'.xlsx')
            #obs_dfs[obs].set_index('freq',inplace=True)

            

#clear some variables
files_raw=None
files=None
filelist=None

In [3]:
#fix the observation names (keys for dictionary obs_dfs) to be the MJD. Will be used for the time in the final plot.
dates=[]
for i in range(0,len(observations)):
    obs=observations[i]
    strt = obs.find('.',obs.find('.')+1)
    dates.append(obs[strt+1:strt+6])
dates=np.sort(dates)

In [4]:
#only take %flagged column
for obs in observations:
    indexcol=obs_dfs[obs].index.values
    obs_dfs[obs]=pd.DataFrame(list(obs_dfs[obs]['%flagged']),index=indexcol)

In [5]:
#create dataframe of all data
big_df = pd.concat(obs_dfs,axis=1)
big_df=big_df.transpose()
big_df.index=dates
big_df
#frequencies 949.2 -> 1439.184375 in 31360 columns

,949.2,949.215625,949.23125,949.246875,949.2625,949.278125,949.29375,949.309375,949.325,949.340625,...,1439.04375,1439.059375,1439.075,1439.090625,1439.10625,1439.121875,1439.1375,1439.153125,1439.16875,1439.184375
57525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.965095,0.971151,0.975565,0.981428,0.985619,0.988587,0.991208,0.992628,1.0,1.0
57527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0
57535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57536,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57538,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57539,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0
57565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#change NaNs to 1.0
big_arr=np.array(big_df)
for i in range(0,big_df.shape[0]):
    for j in range(0,big_df.shape[1]):
        if math.isnan(big_arr[i,j]):
            big_arr[i,j]=1.0
fixed_df=pd.DataFrame(big_arr)
fixed_df.columns = big_df.columns

In [7]:
#average every two elements 2x. Make big_df smaller so that seaborn will actually plot it
trunc_list=[]

for mjd in fixed_df.index.values:
    ser = fixed_df.loc[mjd,:]
    #trunc_list.append(ser.rolling(window=5).mean())
    avgs_2 = [(a + b) / 2 for a, b in zip(*[iter(list(ser))]*2)]
    avgs_4 = [(a + b) / 2 for a, b in zip(*[iter(list(avgs_2))]*2)]
    #avgs_8 = [(a + b) / 2 for a, b in zip(*[iter(list(avgs_4))]*2)]
    trunc_list.append(avgs_4)
    
trunc_cols=list(fixed_df.columns.values)[::4]
trunc_df=pd.DataFrame(trunc_list,index=dates,columns=trunc_cols)

In [8]:
sns.heatmap(trunc_df)

In [8]:
#save dataframes if desired
#writer=pd.ExcelWriter('phase3_waterfall_full.xlsx',engine='xlsxwriter')
#fixed_df.to_excel(writer, sheet_name='Sheet1')
#writer.save()

fixed_df.to_csv('phase3_waterfall_full.csv')

writer=pd.ExcelWriter('phase3_waterfall_trun.xlsx',engine='xlsxwriter')
trunc_df.to_excel(writer, sheet_name='Sheet1')
writer.save()

----

In [9]:
fixed_df

,949.2,949.215625,949.23125,949.246875,949.2625,949.278125,949.29375,949.309375,949.325,949.340625,...,1439.04375,1439.059375,1439.075,1439.090625,1439.10625,1439.121875,1439.1375,1439.153125,1439.16875,1439.184375
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.965095,0.971151,0.975565,0.981428,0.985619,0.988587,0.991208,0.992628,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [11]:
strts=[]
for obs in observations:
    strts.append(obs_dfs[obs].index.values.min())
strts

[959.20000000000005,
 959.20000000000005,
 954.20000000000005,
 954.20000000000005,
 954.20000000000005,
 949.20000000000005,
 949.20000000000005,
 959.20000000000005,
 954.20000000000005]